In [6]:
import string 
import seaborn as sns
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
import warnings
warnings.filterwarnings("ignore")
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd
import io

In [7]:
rstate=10

In [8]:
df = pd.read_csv(r"G:\OneDrive - northsouth.edu\CODES\PROJECTS\PROJECT-CSE299-CYBERBULLYING PREVENTION-DETECTION and MENTAL COUNSELING SYSTEM\Machine Learning Models\bengali_hate_v2.0.csv")


In [9]:
def remove_punctuation_from_sentence(sentence):
    sentence_without_punctuation="".join([c for c in sentence if c not in string.punctuation])
    return sentence_without_punctuation

In [10]:
df['sentence without punctuation'] = df['sentence'].apply(lambda x:remove_punctuation_from_sentence(x))


In [11]:
from bnlp import BasicTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bnlp.corpus.util import remove_stopwords

In [12]:
basic_tokenizer = BasicTokenizer()
df['tokenized_sentence'] = df['sentence without punctuation'].apply(lambda x: basic_tokenizer.tokenize(x))
df.head()

,sentence,category,hate,sentence without punctuation,tokenized_sentence
0,বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না - পোল...,Personal,0,বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না পোলা...,"[বৌদির, দুধ, দেকে, তো, আমার, ই, চোখ, ঠিক, ছিলো..."
1,এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে ...,Political,1,এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে ...,"[এই, সরকার, কে, যারা, নির্লজ্জের, মত, সাপোর্ট,..."
2,পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...,Geopolitical,3,পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...,"[পিলখানা, হত্যাকান্ড, বাংলাদেশের, প্রতিরক্ষা, ..."
3,ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভা...,Geopolitical,3,ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভা...,"[ভারতের, অর্থনীতি, নিয়ে, আপনাদের, ভাবতে, হবে,..."
4,খানকির পুলা মালায়নদের মেরে সাফা করে ফেল,Personal,0,খানকির পুলা মালায়নদের মেরে সাফা করে ফেল,"[খানকির, পুলা, মালায়নদের, মেরে, সাফা, করে, ফেল]"


In [13]:
df['Removed Stopped word'] = df['sentence without punctuation'].apply(lambda x: remove_stopwords(x,stopwords))


In [14]:
df['hate_speech']=df['Removed Stopped word'].apply(' '.join)

In [15]:
x=df['hate_speech']
y=df['hate']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=rstate)

In [17]:
cv = CountVectorizer(stop_words = stopwords)
cv.fit(x)

# transform the training and testing data using count vectorizer object
X_train_count =  cv.transform(X_train)
X_test_count =  cv.transform(X_test)

#Tfid transformer
tfidf_tr = TfidfTransformer()
X_train_count_tfidf = tfidf_tr.fit_transform(X_train_count)
X_test_count_tfidf = tfidf_tr.transform(X_test_count)

# ngram level tf-idf 
tv_ngram = TfidfVectorizer(ngram_range=(1,2))
tv_ngram.fit(x)
X_train_tfidf_ngram =  tv_ngram.transform(X_train)
X_test_tfidf_ngram =  tv_ngram.transform(X_test)

In [18]:
def classifierModel(classifier, Xtrain, Ytrain, Xtest,roc):
    classifier.fit(Xtrain, Ytrain)
    predictions = classifier.predict(Xtest)
    print(30*'=')
    print(f"Accuracy : {accuracy_score(predictions, y_test)*float(100)}%")
    print(30*'=')
    if roc:
        probs = classifier.predict_proba(X_test_count_tfidf)
        preds = probs[:,1]
        fpr_svm, tpr_svm, threshold_svm = metrics.roc_curve(y_test, preds)
        roc_auc_svm = metrics.auc(fpr_svm, tpr_svm)

        import matplotlib.pyplot as plt
        plt.title(type(classifier).__name__)
        plt.plot(fpr_svm, tpr_svm, 'b', label = 'AUC = %0.2f' % roc_auc_svm)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()
    
    target_names = ['class 0', 'class 1']
    print(classification_report(y_test, predictions, target_names=target_names))


In [19]:
from xgboost import XGBClassifier

print("\nXGBOOST with Count Vectors: ")
classifierModel(XGBClassifier(), X_train_count, y_train, X_test_count,roc=True)

print("\nXGBOOST with Count Vectors + TF-IDF: ")
classifierModel(XGBClassifier(), X_train_count_tfidf, y_train, X_test_count_tfidf,roc=True)

print("\nXGBOOST with N-Gram Vectors: ")
classifierModel(XGBClassifier(), X_train_tfidf_ngram, y_train, X_test_tfidf_ngram,roc=False)


XGBOOST with Count Vectors: 
Accuracy : 69.03508771929825%


ValueError: multiclass format is not supported